In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tensorflow as tf
import random
import json
import pickle

from time import sleep


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
with open("intents.json") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
    for intent in data ["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

In [ ]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
model = models.Sequential([
    layers.InputLayer(input_shape=(len(training[0]),)),
    layers.Dense(8, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(len(output[0]), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Check if the model file exists
model_filepath = "model.h5"

if tf.io.gfile.exists(model_filepath):
    # Load the pre-trained model
    model = tf.keras.models.load_model(model_filepath)
else:
    # Train the model
    model.fit(training, output, epochs=200, batch_size=8, verbose=1)

    # Save the trained model
    model.save(model_filepath)


Train on 217 samples
Epoch 1/200
217/217 [==============================] - 0s 1ms/sample - loss: 3.8888 - acc: 0.0415
Epoch 2/200
217/217 [==============================] - 0s 164us/sample - loss: 3.8772 - acc: 0.0691
Epoch 3/200
217/217 [==============================] - 0s 166us/sample - loss: 3.8651 - acc: 0.0829
Epoch 4/200
217/217 [==============================] - 0s 172us/sample - loss: 3.8506 - acc: 0.0829
Epoch 5/200
217/217 [==============================] - 0s 162us/sample - loss: 3.8333 - acc: 0.0876
Epoch 6/200
217/217 [==============================] - 0s 160us/sample - loss: 3.8119 - acc: 0.0876
Epoch 7/200
217/217 [==============================] - 0s 249us/sample - loss: 3.7827 - acc: 0.0922
Epoch 8/200
217/217 [==============================] - 0s 362us/sample - loss: 3.7486 - acc: 0.0876
Epoch 9/200
217/217 [==============================] - 0s 414us/sample - loss: 3.7062 - acc: 0.0876
Epoch 10/200
217/217 [==============================] - 0s 380us/sample - loss: 3

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)


In [ ]:
def chat():
    print("Hi, How can i help you ?")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([numpy.reshape(bag_of_words(inp, words), (1, -1))])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        if results[results_index] > 0.8:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            Bot = random.choice(responses)
            print(Bot)
        else:
            print("I don't understand!")
chat()

Hi, How can i help you ?
You: hi


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Hello
You: what is your name
I'm Joana!
You: how are you
There are various online platforms for education. Let me know your interests, and I can suggest suitable resources.
You: tell me advise
I don't understand!
You: tell me some news 
I don't understand!
You: quit
